# Final Project: Functional Programming

**Laura Belizón Merchán** - 100452273  
**Jorge Lázaro Ruiz** - 100452172  
*Degree in Applied Mathematics and Computation*

## Brief summary

*Pending...*

In [ ]:
-- Necessary imports
import Data.Maybe (fromJust)
import Data.List (intercalate, find)
import Data.Time
import Data.Time.Calendar.WeekDate

## Types

> 1. Types to represent all the former concepts. An exhaustive type definition is required, use the most appropriate way (type, data, newtype) for each of them.

An agenda is described as a list of events, so it makes sense to center the type definitions around the characteristics of an event.

For this, we can create the different types of descriptors defined and then use them for the constructor of an event.

In [ ]:
type Name = String
data Type = Personal | Health | Work | Management
data Date = Date {day :: Int, month :: String, year :: Int, workday :: Bool}
data Time = Time {start :: (Int, Int), end :: (Int, Int), duration :: Maybe Int}
newtype Participants = Participants [Name]
data Repetitions = Punctual | Daily Int | Weekday Int | Weekly Int

* `Name` is just an alias for a `String`, as there are no spectial requirements for it.
* `Type` and `Repetitions` need to be instances of class `Show` if we want to be able to print them, and since they can only take a set of values we must define them as a `data`.
* `Date` and `Time` require a constructor with multiple parameters, hence we use `data` to define them. They also have to be instances of class `Show` so we can pretty-print them.
* `Participants` could be defined as an alias for `[Name]` but in order to avoid confusion with just any other list of strings, we decided to make it into a `newtype` to distinguish it and prevent unintended operations with other lists of strings.

Because we will need to display this to the user, we declare each type as a `Show` instance and define `show` for each one.

In [ ]:
instance Show Type where
    show Personal = "Personal"
    show Health = "Health"
    show Work = "Work"
    show Management = "Management"

instance Show Date where
    show (Date day month year workday) = show day ++ " " ++ month ++ " " ++ show year ++ " " ++
        if workday then "(Workday)" else ""

leadingZero :: Int -> String
leadingZero n = if n < 10 then "0" ++ show n else show n

instance Show Time where
    show (Time (startHour, startMinute) (endHour, endMinute) duration) = 
        show startHour ++ ":" ++ leadingZero startMinute ++ " - " ++ 
        show endHour ++ ":" ++ leadingZero endMinute ++ 
        maybe "" (\d -> "  (Duration: " ++ show d ++ " minutes)") duration

instance Show Participants where
    show (Participants names) = intercalate ", " names

instance Show Repetitions where
    show Punctual = "Punctual"
    show (Daily n) = "Daily " ++ show n
    show (Weekday n) = "Weekday " ++ show n
    show (Weekly n) = "Weekly " ++ show n

Because we will need to compare dates and times, we declare each one as an `Eq` instance and define `==` for each one.

In [ ]:
instance Eq Date where
    (Date day1 month1 year1 _) == (Date day2 month2 year2 _) = 
        day1 == day2 && month1 == month2 && year1 == year2

instance Eq Time where
    (Time (startHour1, startMinute1) (endHour1, endMinute1) _) == 
        (Time (startHour2, startMinute2) (endHour2, endMinute2) _) = 
            startHour1 == startHour2 && startMinute1 == startMinute2 && 
            endHour1 == endHour2 && endMinute1 == endMinute2

With all the types defined, all we need to do is create the type `Descriptor`, with a constructor for each field.  
After that, we can define `Event` as a list of `Descriptor`s.

In [ ]:
data Descriptor =
    NameDescriptor Name
    | TypeDescriptor Type
    | DateDescriptor Date
    | TimeDescriptor Time
    | ParticipantsDescriptor Participants
    | RepetitionsDescriptor Repetitions
    deriving Show

type Event = [Descriptor]

## Event display

> 2. A function to show the events in a pretty way, following the order of descriptors as presented above.

Because the `Event` type is just a list of `Descriptor`s with no particular order, we need to sort it before printing it.

For this, we created a set of "getter" functions that find instances of a given descriptor in a list of descriptors.

In [ ]:
getName :: Event -> Maybe Name
getName event = case find isNameDescriptor event of
        Just (NameDescriptor name) -> Just name
        _ -> Nothing
    where
        isNameDescriptor (NameDescriptor _) = True
        isNameDescriptor _ = False

getType :: Event -> Maybe Type
getType event = case find isTypeDescriptor event of
        Just (TypeDescriptor type') -> Just type'
        _ -> Nothing
    where
        isTypeDescriptor (TypeDescriptor _) = True
        isTypeDescriptor _ = False

getDate :: Event -> Maybe Date
getDate event = case find isDateDescriptor event of
        Just (DateDescriptor date) -> Just date
        _ -> Nothing
    where
        isDateDescriptor (DateDescriptor _) = True
        isDateDescriptor _ = False

getTime :: Event -> Maybe Time
getTime event = case find isTimeDescriptor event of
        Just (TimeDescriptor time) -> Just time
        _ -> Nothing
    where
        isTimeDescriptor (TimeDescriptor _) = True
        isTimeDescriptor _ = False

getParticipants :: Event -> Maybe Participants
getParticipants event = case find isParticipantsDescriptor event of
        Just (ParticipantsDescriptor participants) -> Just participants
        _ -> Nothing
    where
        isParticipantsDescriptor (ParticipantsDescriptor _) = True
        isParticipantsDescriptor _ = False

getRepetitions :: Event -> Maybe Repetitions
getRepetitions event = case find isRepetitionsDescriptor event of
        Just (RepetitionsDescriptor repetitions) -> Just repetitions
        _ -> Nothing
    where
        isRepetitionsDescriptor (RepetitionsDescriptor _) = True
        isRepetitionsDescriptor _ = False

In [ ]:
printEvent :: Event -> IO ()
printEvent event = do
    putStrLn "Event:"
    putStrLn $ "  Name: " ++ maybe "Unknown" show (getName event)
    putStrLn $ "  Type: " ++ maybe "Unknown" show (getType event)
    putStrLn $ "  Date: " ++ maybe "Unknown" show (getDate event)
    putStrLn $ "  Time: " ++ maybe "Unknown" show (getTime event)
    putStrLn $ "  Participants: " ++ maybe "Unknown" show (getParticipants event)
    putStrLn $ "  Repetitions: " ++ maybe "Unknown" show (getRepetitions event)

-- Example usage
eventName = NameDescriptor "Example"
eventType = TypeDescriptor Work
eventDate = DateDescriptor (Date 6 "December" 2023 True)
eventTime = TimeDescriptor (Time (10, 30) (12, 0) (Just 90))
eventParticipants = ParticipantsDescriptor (Participants ["John", "Alice", "Bob"])
eventRepetitions = RepetitionsDescriptor (Weekly 2)
exampleEvent = [eventName, eventType, eventDate, eventTime, eventParticipants, eventRepetitions]
printEvent exampleEvent

## Utility functions

> 3. An `isWorkingDate Date [Date]` function that receives a `Date` and a list of holidays and returns if the day is weekday or weekend/holiday by considering leap years and the fact that 1<sup>st</sup> January 2022 was Saturday.

To do this, we can use `Data.Time.Calendar`'s `dayOfWeek` function to get the day of the week of a given date.

Since our `Date` type is different from the `Day` in the module, we need to convert it using a custom `toDay` function.

Then, we can check if the date is a Saturday or Sunday, and if it is, return `False`.

In [ ]:
toDay :: Date -> Day
toDay (Date d m y _) = fromGregorian (toInteger y) (monthToInt m) d

monthToInt :: String -> Int
monthToInt month = fromJust $ lookup month (zip months [1..12])
    where months = ["January", "February", "March", "April", "May", "June", "July", "August", "September",
                    "October", "November", "December"]

isWeekend :: Date -> Bool
isWeekend date =
  let (_, _, dayOfWeek) = toWeekDate $ toDay date
  in dayOfWeek == 6 || dayOfWeek == 7  -- 6 is Saturday, 7 is Sunday

isWorkingDate :: Date -> [Date] -> Bool
isWorkingDate date holidays
    | date `elem` holidays = False
    | otherwise = not $ isWeekend date

-- Example usage
exampleDate = Date 25 "December" 2023 True
exampleHolidays = [Date 25 "December" 2023 False, Date 1 "January" 2024 False]
print $ isWorkingDate exampleDate exampleHolidays
